In [1]:
import requests

URL = "https://s199-de.ogame.gameforge.com/api/universe.xml"

response = requests.get(URL)
with open('universe.xml', 'wb') as file:
    file.write(response.content)

    

In [2]:
import csv 
import xml.etree.ElementTree as ET 

tree = ET.parse('universe.xml')

In [3]:
root = tree.getroot()

In [4]:
open_coords = [f"{g}:{s}:8" for g in range(1, 9) for s in range(1, 500) ]
for planet in root.findall('planet'):
    coords = planet.get("coords")
    if coords[-1] == "8":
        open_coords.remove(coords)
open_coords

['8:387:8']

In [5]:
timestamp = int(root.get('timestamp'))
timestamp

1718104434

In [6]:
from datetime import datetime
print(datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))


2024-06-11 11:13:54


In [3]:
import requests
import xml.etree.ElementTree as ET 
 
url = f"https://s199-de.ogame.gameforge.com/api/highscore.xml?category=1&type=1"
response = requests.get(url)
filename = 'highscore.xml'
with open(filename, 'wb') as file:
    file.write(response.content)
tree = ET.parse(filename)
root = tree.getroot()

In [4]:
root

<Element 'highscore' at 0x7f6a1d6f6070>

In [ ]:
url = f"https://s199-de.ogame.gameforge.com/api/players.xml"
response = requests.get(url)
filename = 'players.xml'
with open(filename, 'wb') as file:
    file.write(response.content)

In [6]:
url = f"https://s199-de.ogame.gameforge.com/api/alliances.xml"
response = requests.get(url)
filename = 'alliances.xml'
with open(filename, 'wb') as file:
    file.write(response.content)

In [23]:
import pandas as pd

/home/beni/ogame-galaxy-webhook/.venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [24]:
highscore = pd.read_xml("highscore.xml")
players = pd.read_xml("players.xml")
alliances = pd.read_xml("alliances.xml")

In [25]:
highscore.head(1)

,position,id,score
0,1,110961,274903885243


In [26]:
alliance_id = alliances[(alliances.name == "La Familia") | (alliances.tag == "L-F")].id.iloc[0]

In [27]:
players = players[players.alliance == alliance_id].join(highscore.set_index("id"), on="id")
players

,id,name,status,alliance,position,score
10,100054,ToxicAttack,vI,500683.0,434,318741377
399,107234,Pfandflaschensammler,None,500683.0,586,153344335
657,109411,DcBen,None,500683.0,62,2733333138
679,109496,Estobar,None,500683.0,97,2125693847
758,109702,Madbutcher,vi,500683.0,117,1831903106
785,109747,Melkor,None,500683.0,8,10767107968
804,109773,Ynsane,None,500683.0,9,10747048401
939,109976,Hui Buh,v,500683.0,282,740406406
948,109993,Manticore,None,500683.0,109,1898494064
958,110009,Babs,None,500683.0,360,478865676


In [28]:
players["date"] = pd.Timestamp.today().date()
players = players.drop(["id", "alliance"], axis=1)

In [ ]:
players.to_csv("highscore.csv", mode="a", header=None, index=None)

In [30]:
from functions import to_md
players

,name,status,position,score,date
10,ToxicAttack,vI,434,318741377,2025-07-09
399,Pfandflaschensammler,None,586,153344335,2025-07-09
657,DcBen,None,62,2733333138,2025-07-09
679,Estobar,None,97,2125693847,2025-07-09
758,Madbutcher,vi,117,1831903106,2025-07-09
785,Melkor,None,8,10767107968,2025-07-09
804,Ynsane,None,9,10747048401,2025-07-09
939,Hui Buh,v,282,740406406,2025-07-09
948,Manticore,None,109,1898494064,2025-07-09
958,Babs,None,360,478865676,2025-07-09


In [35]:
table_text = players[["name", "position", "score"]].to_string(index=False)

# Wrap with Discord code block
message = f"```{table_text}```"

In [ ]:
from discord import SyncWebhook
webhook = SyncWebhook.from_url("webhook.url")
webhook.send(message)

In [ ]:
import requests
url = f"https://www.mmorpg-stat.eu/base.php?ftr=500683.all&univers=_199&pays=1&statview=3"
response = requests.get(url)
filename = 'alliance.html'
with open(filename, 'wb') as file:
    file.write(response.content)

In [ ]:
from bs4 import BeautifulSoup
with open("alliance.html") as file:
    html = BeautifulSoup(file)

In [ ]:
table = html.findAll("table")[-1]
table

/tmp/ipykernel_13783/3190337590.py:1: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = html.findAll("table")[-1]


<table border="0" cellpadding="0" cellspacing="0" class="table table-dark ess essa tab_classement" id="essailive"><thead><colgroup><col span="4"/><col style="background-color: rgba(20, 20, 20, 0.3);"/></colgroup><tr><th scope="col" style="width:10px;text-align: center">#<div id="returnOnTop"></div></th><th scope="col" style="width:10px;text-align: right">Position</th><th scope="col" style="text-align: left">Player</th><th scope="col" style="text-align: left">Alliance</th><th scope="col" style="text-align: right">Points</th><th scope="col" style="text-align: right">Difference</th></tr></thead><tbody>
<form action="base.php?pagex=6&amp;univers=_199&amp;pays=1&amp;calag=1" method="post" name="form_compare1">
<tr><td align="center" class="" nowrap="" style=" white-space:nowrap;vertical-align:middle;"> 1</td><td align="right" class="" nowrap="" style=" white-space:nowrap;vertical-align:middle">7</td><td align="left" class="" nowrap="" style=" white-space:nowrap;vertical-align:middle"><span>

In [ ]:
with open("table.html", mode="w+") as file:
    file.write(str(table))

In [155]:
import pandas as pd
history = pd.read_csv("highscore.csv")
history

,name,status,position,score,date
0,ToxicAttack,vI,434,31871377,2025-07-08
1,Pfandflaschensammler,NaN,586,15334435,2025-07-08
2,DcBen,NaN,62,27333533138,2025-07-08
3,Estobar,NaN,97,2125693847,2025-07-08
4,Madbutcher,vi,117,1831903106,2025-07-08
...,...,...,...,...,...
109,Body Snatcher,v,621,134533702,2025-07-09
110,EsHeR,v,193,1189409840,2025-07-09
111,Thunder,NaN,84,2358993664,2025-07-09
112,Lost,NaN,310,611497763,2025-07-09


In [156]:
history = history.sort_values(["name", "date"])

In [157]:
history = history.join(history[["position", "score"]].shift(), rsuffix="_prev")

In [158]:
for col in ["score", "position"]:
    history[f"{col}_diff"] = history[col]-history[f"{col}_prev"]
history["score_rel"] = (history["score"]-history["score_prev"]) / history["score_prev"] * 100

In [159]:
history

,name,status,position,score,date,position_prev,score_prev,score_diff,position_diff,score_rel
12,Accur8,vI,89,2262812311,2025-07-08,NaN,NaN,NaN,NaN,NaN
31,Accur8,vI,89,2262812311,2025-07-09,89.0,2.262812e+09,0.0,0.0,0.000000
50,Accur8,vI,89,2262812311,2025-07-09,89.0,2.262812e+09,0.0,0.0,0.000000
69,Accur8,vI,9,2262812311,2025-07-09,89.0,2.262812e+09,0.0,-80.0,0.000000
88,Accur8,vI,89,2262812311,2025-07-09,9.0,2.262812e+09,0.0,80.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
32,m0sT,NaN,29,4618263930,2025-07-09,29.0,4.618264e+09,0.0,0.0,0.000000
51,m0sT,NaN,29,4618266292,2025-07-09,29.0,4.618264e+09,2362.0,0.0,0.000051
70,m0sT,NaN,29,4618266292,2025-07-09,29.0,4.618266e+09,0.0,0.0,0.000000
89,m0sT,NaN,29,4618266292,2025-07-09,29.0,4.618266e+09,0.0,0.0,0.000000


In [160]:
history = history.groupby("name").last().reset_index()
history = history.sort_values("score_rel", ascending=False)

In [161]:
history

,name,status,position,score,date,position_prev,score_prev,score_diff,position_diff,score_rel
14,Stefen,v,202,1130007119,2025-07-09,2002.0,1.100071e+08,1.020000e+09,-1800.0,927.212720
15,Thunder,None,84,2358993664,2025-07-09,84.0,2.358962e+09,3.161800e+04,0.0,0.001340
18,m0sT,None,29,4618285629,2025-07-09,29.0,4.618266e+09,1.933700e+04,0.0,0.000419
12,Melkor,None,8,10767793301,2025-07-09,8.0,1.076777e+10,2.348400e+04,0.0,0.000218
17,Ynsane,None,9,10747078610,2025-07-09,9.0,1.074706e+10,2.000900e+04,0.0,0.000186
6,Estobar,None,97,2125719680,2025-07-09,97.0,2.125717e+09,2.420000e+03,0.0,0.000114
0,Accur8,vI,89,2262812311,2025-07-09,89.0,2.262812e+09,0.000000e+00,0.0,0.000000
2,Body Snatcher,v,621,134533702,2025-07-09,21.0,1.345337e+08,0.000000e+00,600.0,0.000000
1,Babs,None,360,479165844,2025-07-09,360.0,4.791658e+08,0.000000e+00,0.0,0.000000
4,DcBen,None,62,2733333138,2025-07-09,62.0,2.733333e+09,0.000000e+00,0.0,0.000000


In [162]:
history["position_diff"] = history["position_diff"].apply(lambda x: f"{x:.0f}")

In [163]:
history["score"] = history["score"].apply(lambda x: f"{x:,}")
history["score_diff"] = history["score_diff"].apply(lambda x: f"{x:,}")

In [164]:
history["score_rel"] = history["score_rel"].apply(lambda x: f"{x:.2f}%")

In [165]:
history

,name,status,position,score,date,position_prev,score_prev,score_diff,position_diff,score_rel
14,Stefen,v,202,"1,130,007,119",2025-07-09,2002.0,1.100071e+08,"1,020,000,000.0",-1800,927.21%
15,Thunder,None,84,"2,358,993,664",2025-07-09,84.0,2.358962e+09,"31,618.0",0,0.00%
18,m0sT,None,29,"4,618,285,629",2025-07-09,29.0,4.618266e+09,"19,337.0",0,0.00%
12,Melkor,None,8,"10,767,793,301",2025-07-09,8.0,1.076777e+10,"23,484.0",0,0.00%
17,Ynsane,None,9,"10,747,078,610",2025-07-09,9.0,1.074706e+10,"20,009.0",0,0.00%
6,Estobar,None,97,"2,125,719,680",2025-07-09,97.0,2.125717e+09,"2,420.0",0,0.00%
0,Accur8,vI,89,"2,262,812,311",2025-07-09,89.0,2.262812e+09,0.0,0,0.00%
2,Body Snatcher,v,621,"134,533,702",2025-07-09,21.0,1.345337e+08,0.0,600,0.00%
1,Babs,None,360,"479,165,844",2025-07-09,360.0,4.791658e+08,0.0,0,0.00%
4,DcBen,None,62,"2,733,333,138",2025-07-09,62.0,2.733333e+09,0.0,0,0.00%
